In [ ]:
!pip install torch transformers
!pip install -U datasets
!git clone https://github.com/davidjurgens/citation-function

In [2]:
import os
with open('api_openrouter.txt', 'r') as f:
  API=f.read()


In [ ]:
from datasets import load_dataset
ds = load_dataset("hrithikpiyush/acl-arc")
ds.set_format("pandas")
train=ds["train"][:]
validation=ds["validation"][:]
test=ds["test"][:]

In [ ]:
for text,intent in zip(test['text'],test['intent']):
    print('You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:'
            + 'From the following list: 0:BACKGROUND \n 1:USES, 2:COMPARES OR CONTRASTS, 3:MOTIVATION, 4:CONTINUATION, 5:FUTURE\n')
    print(text+'\n\n')

In [6]:
dico_rez={
    0:[],
    1:[],
    2:[],
    3:[],
    4:[],
    5:[],
}

In [7]:
PROMPT=("""You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation into one of these categories.Think as much as you want, but answer only with the associated number:
          "BACKGROUND":"to provide relevant information for this domain (BACKGROUND).",
          "MOTIVATION":"to illustrate need for data, goals, methods, etc.(MOTIVATION)",
          "USES":" to use data, methods, etc from the citation. (USES) ",  
          "CONTINUATION":"to extend the citation's data, methods, etc. (CONTINUATION)",
          "COMPARES OR CONTRASTS":"to express similarity/differences to the citation (COMPARES OR CONTRASTS)",
          "FUTURE": "to be a potential avenue for future work (FUTURE)"
          
          ANSWER WITH THE CORRECT ASSOCIATED NUMBER: 0:BACKGROUND \n 1:USES \n 2:COMPARES OR CONTRASTS \n 3:MOTIVATION \n 4:CONTINUATION \n 5:FUTURE\n"""+ text+'\n\n')

In [ ]:
import requests
import json
import os

les_rez=[]
dico_rez={0:[],1:[],2:[],3:[],4:[],5:[]}

url = "https://openrouter.ai/api/v1/chat/completions"


headers = {
  "Authorization": f"Bearer {API}",
  "Content-Type": "application/json"
}


for text,intent in zip(test['cleaned_cite_text'],test['intent']):
  PROMPT=("""You are an expert in scientometry, your role is to classify the intent of a citation in a scientific paper. The possible intents are: Background, Uses, Extends, Motivation, CompareOrContrast, Future, Other. Given the text of the citation, classify the intent of the citation @@CITATION into one of these categories.Think as much as you want, in the end answer only with the associated number:
          "BACKGROUND":"to provide relevant information for this domain (BACKGROUND).",
          "MOTIVATION":"to illustrate need for data, goals, methods, etc.(MOTIVATION)",
          "USES":" to use data, methods, etc from the citation. (USES) ",  
          "CONTINUATION":"to extend the citation's data, methods, etc. (CONTINUATION)",
          "COMPARES OR CONTRASTS":"to express similarity/differences to the citation (COMPARES OR CONTRASTS)",
          "FUTURE": "to be a potential avenue for future work (FUTURE)"
          Please reason step by step, and put your final answer within \\boxed{} \\n"""+ text+'\n\n')
  try:
    payload = {
        "model": "deepseek/deepseek-r1-0528",
      "messages": [
        {
          "role": "user",
          "content": PROMPT
        }
      ]
        ,
    #   "provider": {
    #     "allow_fallbacks": True,
    #     "quantizations": [
    #       "fp8"
    #     ]
    # },
        "reasoning": {

      "enabled": True  # Use high reasoning effort

  }
    }
    response = requests.post(url, headers=headers, json=payload)
    les_rez.append(response.json())
    dico_rez[int(intent)].append(response.json())
  except:
    print("error")
    les_rez.append('error '+text)
    continue
  print(response.json())

In [12]:
len(les_rez)

137

In [25]:
BACKUP=les_rez.copy()

In [33]:
les_rez[107]

'error'

In [31]:
les_rez.insert(107,'error')

In [ ]:
les_rez[105]['id']

In [23]:
les_rez[106]

'error'

In [21]:
for i in range(0,len(les_rez)):
    try:
        if les_rez[i]['id'] == 'gen-1760362894-5kkXe3tlsxqDLVU3OAGV':
            print(i)
    except:
        pass

106


In [34]:
with open(f'DATASET10/results.txt', 'a+',encoding='utf-8') as f:
    for elt in les_rez:
        try:
            f.write(elt['choices'][0]['message']['content'] + '\n\n')
        except:
            f.write('error')

#dump the json from les_rez
with open(f'DATASET10/results.json', 'w+',encoding='utf-8') as f:
    json.dump(les_rez, f, ensure_ascii=False, indent=4)


In [ ]:
for i in range(0,6):
    for elt in dico_rez[i]:
        with open(f'DATASET7/results_openrouter_intent_{i}.txt', 'a+',encoding='utf-8') as f:
            f.write(elt['choices'][0]['message']['content'] + '\n\n')

In [7]:
with open("DATASET7/dico.json", "w+", encoding="utf-8") as f:
    json.dump(dico_rez, f, ensure_ascii=False, indent=4)